In [1]:
'''
!pip install Cython
!git clone https://github.com/waleedka/coco.git

!pip install -U setuptools
!pip install -U wheel
!make install -C coco/PythonAPI
'''

In [2]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
import imageio
import datetime
import tensorflow as tf
import keras
from keras.callbacks import LearningRateScheduler



# Root directory of the project
ROOT_DIR = os.path.abspath("../")
# Ignore Warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
print(ROOT_DIR)
from mrcnn import utils
import mrcnn.model_transformer as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/interior/"))  # To find local version
import interior_transformer

%matplotlib inline 
%load_ext tensorboard

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

/home/julius/Projects/MVOD/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/julius/Projects/MVOD/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/julius/Projects/MVOD/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/julius/Projects/MVOD/lib/python3.6/site-packages/tensorflow/pyth

/home/julius/Projects/MVOD/mulit_view_object_detection
The tensorboard module is not an IPython extension.


In [3]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


In [4]:
class_names = ['BG', 'wall', 'floor', 'cabinet', 'bed', 'chair', 'sofa',
               'table', 'door', 'window', 'bookshelf', 'picture', 'counter',
               'blinds', 'desk', 'shelves', 'curtain', 'dresser', 'pillow', 
               'mirror', 'floor', 'clothes', 'ceiling', 'books', 'refridgerator',
               'television', 'paper', 'towel', 'shower', 'box', 'whiteboard', 
               'person', 'night', 'toilet', 'sink', 'lamp', 'bathtub', 'bag', 
               'otherstructure', 'otherfurniture', 'otherprop']

selected_classes = ['BG', 'cabinet', 'bed', 'chair', 'sofa', 'table', 'window', 
                    'picture', 'blinds', 'shelves', 'curtain', 'dresser', 'pillow', 
                    'mirror',  'clothes','books', 'refrigerator','television', 'paper', 'towel',
                    'toilet', 'sink', 'lamp', 'bathtub', 'bag']
# without curtain
selected_classes = ['BG', 'cabinet', 'bed', 'chair', 'sofa', 'table', 
                    'picture', 'blinds', 'shelves', 'dresser', 'pillow', 
                    'mirror',  'clothes','books', 'refrigerator','television', 'paper', 'towel',
                    'toilet', 'sink', 'lamp', 'bathtub', 'bag']
print(class_names[15])

shelves


In [5]:
NYU40_to_sel_map = {}
selected_class_list = []

for i in range(len(class_names)):
    if class_names[i] in selected_classes:
        NYU40_to_sel_map[i] = selected_classes.index(class_names[i])
        selected_class_list.append(i)
    else:
        NYU40_to_sel_map[i] = 0
NYU40_to_sel_map[class_names.index('desk')] = selected_classes.index('table')
NYU40_to_sel_map[class_names.index('bookshelf')] = selected_classes.index('shelves')
NYU40_to_sel_map[class_names.index('refridgerator')] = selected_classes.index('refrigerator')
selected_class_list.append(class_names.index('desk'))
selected_class_list.append(class_names.index('bookshelf'))
selected_class_list.append(class_names.index('refridgerator'))

"""
NYU40_to_sel_map = {'BG': 'BG', 'wall': 'BG', 'floor': 'BG', 'cabinet': 'cabinet', 
                    'bed': 'bed', 'chair': 'chair', 'sofa': 'sofa', 'table': 'table', 
                    'door': 'BG', 'window': 'window', 'bookshelf': 'shelves', 
                    'picture': 'picture', 'counter': 'BG', 'blinds': 'blinds', 
                    'desk': 'table', 'shelves': 'shelves', 'curtain': 'curtain', 
                    'dresser': 'dresser', 'pillow': 'pillow', 'mirror': 'mirror', 
                    'clothes': 'clothes', 'ceiling': 'BG', 'books': 'books', 
                    'refridgerator': 'refrigerator', 'television': 'television', 
                    'paper': 'paper', 'towel': 'towel', 'shower': 'BG', 'box': 'BG', 
                    'whiteboard': 'BG', 'person': 'BG', 'night': 'BG', 'toilet': 'toilet',
                    'sink': 'sink', 'lamp': 'lamp', 'bathtub': 'bathtub', 'bag': 'bag', 
                    'otherstructure': 'BG', 'otherfurniture': 'BG', 'otherprop': 'BG'}
"""

print(NYU40_to_sel_map)
print(selected_class_list)

{0: 0, 1: 0, 2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 0, 9: 0, 10: 8, 11: 6, 12: 0, 13: 7, 14: 5, 15: 8, 16: 0, 17: 9, 18: 10, 19: 11, 20: 0, 21: 12, 22: 0, 23: 13, 24: 14, 25: 15, 26: 16, 27: 17, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 18, 34: 19, 35: 20, 36: 21, 37: 22, 38: 0, 39: 0, 40: 0}
[0, 3, 4, 5, 6, 7, 11, 13, 15, 17, 18, 19, 21, 23, 25, 26, 27, 33, 34, 35, 36, 37, 14, 10, 24]


In [6]:
#TRAIN_DIR = '/home/erbachj/MaskRCNN/Mask_RCNN/data/InteriorNet/data/HD7/3FO4IDDWIQX7_Dining_room'
#TRAIN_DIR = os.path.join(TRAIN_DIR,'cam0/data/')
TRAIN_DIR = os.path.join(ROOT_DIR, 'data/InteriorNet/data/HD1')
print(TRAIN_DIR)
# Training dataset
dataset_train = interior_transformer.InteriorDataset()
dataset_train.load_Interior(dataset_dir=TRAIN_DIR, subset='train', class_ids=selected_class_list, NYU40_to_sel_map=NYU40_to_sel_map,
                          selected_classes=selected_classes)
dataset_train.prepare()

# Validation dataset
#VAL_DIR = '/home/erbachj/MaskRCNN/Mask_RCNN/data/InteriorNet/data/HD7/3FO4IDEI1LAV_Dining_room'
#VAL_DIR = os.path.join(VAL_DIR,'cam0/data/')
VAL_DIR = os.path.join(ROOT_DIR, 'data/InteriorNet/data/HD1')

dataset_val = interior_transformer.InteriorDataset()
dataset_val.load_Interior(dataset_dir=VAL_DIR, subset='val', class_ids=selected_class_list, NYU40_to_sel_map=NYU40_to_sel_map,
                          selected_classes=selected_classes)
dataset_val.prepare()

/home/julius/Projects/MVOD/mulit_view_object_detection/data/InteriorNet/data/HD1
HD1
/home/julius/Projects/MVOD/mulit_view_object_detection/data/InteriorNet/data/HD1/train/3FO4JXNTQPKL_3
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
3FO4JXNTQPKL_3_id0000000000031666668
3FO4JXNTQPKL_3_id0000000000071666664
3FO4JXNTQPKL_3_id0000000000111666664
3FO4JXNTQPKL_3_id0000000000151666672
3FO4JXNTQPKL_3_id0000000000191666672
3FO4JXNTQPKL_3_id0000000000231666672
3FO4JXNTQPKL_3_id0000000000271666656
3FO4JXNTQPKL_3_id0000000000311666656
3FO4JXNTQPKL_3_id0000000000351666656
3FO4JXNTQPKL_3_id0000000000391666656
3FO4JXNTQPKL_3_id0000000000431666656
3FO4JXNTQPKL_3_id0000000000471666656
3FO4JXNTQPKL_3_id0000000000511666656
3FO4JXNTQPKL_3_id0000000000551666688
3FO4JXNTQPKL_3_id0000000000591666688
3FO4JXNTQPKL_3_id0000000000631666688
3FO4JXNTQPKL_3_id0000000000671666688
3FO4JXNTQPKL_3_id0000000000711666688
3FO4JXNTQPKL_3_id0000000000751666688
3FO4JXNTQPKL_3_id0000000000

3FO4JXNTQPKL_3_id0000000012831666176
3FO4JXNTQPKL_3_id0000000012871666688
3FO4JXNTQPKL_3_id0000000012911666176
3FO4JXNTQPKL_3_id0000000012951666688
3FO4JXNTQPKL_3_id0000000012991666176
3FO4JXNTQPKL_3_id0000000013031666688
3FO4JXNTQPKL_3_id0000000013071666176
3FO4JXNTQPKL_3_id0000000013111666688
3FO4JXNTQPKL_3_id0000000013151666176
3FO4JXNTQPKL_3_id0000000013191666688
3FO4JXNTQPKL_3_id0000000013231666176
3FO4JXNTQPKL_3_id0000000013271666688
3FO4JXNTQPKL_3_id0000000013311666176
3FO4JXNTQPKL_3_id0000000013351666688
3FO4JXNTQPKL_3_id0000000013391666176
3FO4JXNTQPKL_3_id0000000013431666688
3FO4JXNTQPKL_3_id0000000013471666176
3FO4JXNTQPKL_3_id0000000013511666688
3FO4JXNTQPKL_3_id0000000013551666176
3FO4JXNTQPKL_3_id0000000013591666688
3FO4JXNTQPKL_3_id0000000013631666176
3FO4JXNTQPKL_3_id0000000013671666688
3FO4JXNTQPKL_3_id0000000013711666176
3FO4JXNTQPKL_3_id0000000013751666688
3FO4JXNTQPKL_3_id0000000013791666176
3FO4JXNTQPKL_3_id0000000013831666688
3FO4JXNTQPKL_3_id0000000013871666176
3

3FO4JXNTQPKL_3_id0000000023031666688
3FO4JXNTQPKL_3_id0000000023071666176
3FO4JXNTQPKL_3_id0000000023111665664
3FO4JXNTQPKL_3_id0000000023151667200
3FO4JXNTQPKL_3_id0000000023191666688
3FO4JXNTQPKL_3_id0000000023231666176
3FO4JXNTQPKL_3_id0000000023271665664
3FO4JXNTQPKL_3_id0000000023311667200
3FO4JXNTQPKL_3_id0000000023351666688
3FO4JXNTQPKL_3_id0000000023391666176
3FO4JXNTQPKL_3_id0000000023431665664
3FO4JXNTQPKL_3_id0000000023471667200
3FO4JXNTQPKL_3_id0000000023511666688
3FO4JXNTQPKL_3_id0000000023551666176
3FO4JXNTQPKL_3_id0000000023591665664
3FO4JXNTQPKL_3_id0000000023631667200
3FO4JXNTQPKL_3_id0000000023671666688
3FO4JXNTQPKL_3_id0000000023711666176
3FO4JXNTQPKL_3_id0000000023751665664
3FO4JXNTQPKL_3_id0000000023791667200
3FO4JXNTQPKL_3_id0000000023831666688
3FO4JXNTQPKL_3_id0000000023871666176
3FO4JXNTQPKL_3_id0000000023911665664
3FO4JXNTQPKL_3_id0000000023951667200
3FO4JXNTQPKL_3_id0000000023991666688
3FO4JXNTQPKL_3_id0000000024031666176
3FO4JXNTQPKL_3_id0000000024071665664
3

3FO4K2EMJ5OO_id0000000013911666688
3FO4K2EMJ5OO_id0000000013951666176
3FO4K2EMJ5OO_id0000000013991666688
3FO4K2EMJ5OO_id0000000014031666176
3FO4K2EMJ5OO_id0000000014071666688
3FO4K2EMJ5OO_id0000000014111666176
3FO4K2EMJ5OO_id0000000014151666688
3FO4K2EMJ5OO_id0000000014191666176
3FO4K2EMJ5OO_id0000000014231666688
3FO4K2EMJ5OO_id0000000014271666176
3FO4K2EMJ5OO_id0000000014311666688
3FO4K2EMJ5OO_id0000000014351666176
3FO4K2EMJ5OO_id0000000014391666688
3FO4K2EMJ5OO_id0000000014431666176
3FO4K2EMJ5OO_id0000000014471666688
3FO4K2EMJ5OO_id0000000014511666176
3FO4K2EMJ5OO_id0000000014551666688
3FO4K2EMJ5OO_id0000000014591666176
3FO4K2EMJ5OO_id0000000014631666688
3FO4K2EMJ5OO_id0000000014671666176
3FO4K2EMJ5OO_id0000000014711666688
3FO4K2EMJ5OO_id0000000014751666176
3FO4K2EMJ5OO_id0000000014791666688
3FO4K2EMJ5OO_id0000000014831666176
3FO4K2EMJ5OO_id0000000014871666688
3FO4K2EMJ5OO_id0000000014911666176
3FO4K2EMJ5OO_id0000000014951666688
3FO4K2EMJ5OO_id0000000014991666176
3FO4K2EMJ5OO_id00000

3FO4K2EMJ5OO_id0000000024911667200
3FO4K2EMJ5OO_id0000000024951666688
3FO4K2EMJ5OO_id0000000024991666176
3FO4K2EMJ5OO_id0000000025031665664
3FO4K2EMJ5OO_id0000000025071667200
3FO4K2EMJ5OO_id0000000025111666688
3FO4K2EMJ5OO_id0000000025151666176
3FO4K2EMJ5OO_id0000000025191665664
3FO4K2EMJ5OO_id0000000025231667200
3FO4K2EMJ5OO_id0000000025271666688
3FO4K2EMJ5OO_id0000000025311666176
3FO4K2EMJ5OO_id0000000025351665664
3FO4K2EMJ5OO_id0000000025391667200
3FO4K2EMJ5OO_id0000000025431666688
3FO4K2EMJ5OO_id0000000025471666176
3FO4K2EMJ5OO_id0000000025511665664
3FO4K2EMJ5OO_id0000000025551667200
3FO4K2EMJ5OO_id0000000025591666688
3FO4K2EMJ5OO_id0000000025631666176
3FO4K2EMJ5OO_id0000000025671665664
3FO4K2EMJ5OO_id0000000025711667200
3FO4K2EMJ5OO_id0000000025751666688
3FO4K2EMJ5OO_id0000000025791666176
3FO4K2EMJ5OO_id0000000025831665664
3FO4K2EMJ5OO_id0000000025871667200
3FO4K2EMJ5OO_id0000000025911666688
3FO4K2EMJ5OO_id0000000025951666176
3FO4K2EMJ5OO_id0000000025991665664
3FO4K2EMJ5OO_id00000

...

3FO4JXP5VPDL_id0000000035231666176
3FO4JXP5VPDL_id0000000035271667712
3FO4JXP5VPDL_id0000000035311665152
3FO4JXP5VPDL_id0000000035351666688
3FO4JXP5VPDL_id0000000035391668224
3FO4JXP5VPDL_id0000000035431665664
3FO4JXP5VPDL_id0000000035471667200
3FO4JXP5VPDL_id0000000035511664640
3FO4JXP5VPDL_id0000000035551666176
3FO4JXP5VPDL_id0000000035591667712
3FO4JXP5VPDL_id0000000035631665152
3FO4JXP5VPDL_id0000000035671666688
3FO4JXP5VPDL_id0000000035711668224
3FO4JXP5VPDL_id0000000035751665664
3FO4JXP5VPDL_id0000000035791667200
3FO4JXP5VPDL_id0000000035831664640
3FO4JXP5VPDL_id0000000035871666176
3FO4JXP5VPDL_id0000000035911667712
3FO4JXP5VPDL_id0000000035951665152
3FO4JXP5VPDL_id0000000035991666688
3FO4JXP5VPDL_id0000000036031668224
3FO4JXP5VPDL_id0000000036071665664
3FO4JXP5VPDL_id0000000036111667200
3FO4JXP5VPDL_id0000000036151664640
3FO4JXP5VPDL_id0000000036191666176
3FO4JXP5VPDL_id0000000036231667712
3FO4JXP5VPDL_id0000000036271665152
3FO4JXP5VPDL_id0000000036311666688
3FO4JXP5VPDL_id00000

Done (t=0.23s)
creating index...
index created!
3FO4JYBACBKR_id0000000000031666668
3FO4JYBACBKR_id0000000000071666664
3FO4JYBACBKR_id0000000000111666664
3FO4JYBACBKR_id0000000000151666672
3FO4JYBACBKR_id0000000000191666672
3FO4JYBACBKR_id0000000000231666672
3FO4JYBACBKR_id0000000000271666656
3FO4JYBACBKR_id0000000000311666656
3FO4JYBACBKR_id0000000000351666656
3FO4JYBACBKR_id0000000000391666656
3FO4JYBACBKR_id0000000000431666656
3FO4JYBACBKR_id0000000000471666656
3FO4JYBACBKR_id0000000000511666656
3FO4JYBACBKR_id0000000000551666688
3FO4JYBACBKR_id0000000000591666688
3FO4JYBACBKR_id0000000000631666688
3FO4JYBACBKR_id0000000000671666688
3FO4JYBACBKR_id0000000000711666688
3FO4JYBACBKR_id0000000000751666688
3FO4JYBACBKR_id0000000000791666688
3FO4JYBACBKR_id0000000000831666688
3FO4JYBACBKR_id0000000000871666688
3FO4JYBACBKR_id0000000000911666688
3FO4JYBACBKR_id0000000000951666688
3FO4JYBACBKR_id0000000000991666688
3FO4JYBACBKR_id0000000001031666688
3FO4JYBACBKR_id0000000001071666688
3FO4JYB

3FO4JXNTQPKL_id0000000013711666176
3FO4JXNTQPKL_id0000000013751666688
3FO4JXNTQPKL_id0000000013791666176
3FO4JXNTQPKL_id0000000013831666688
3FO4JXNTQPKL_id0000000013871666176
3FO4JXNTQPKL_id0000000013911666688
3FO4JXNTQPKL_id0000000013951666176
3FO4JXNTQPKL_id0000000013991666688
3FO4JXNTQPKL_id0000000014031666176
3FO4JXNTQPKL_id0000000014071666688
3FO4JXNTQPKL_id0000000014111666176
3FO4JXNTQPKL_id0000000014151666688
3FO4JXNTQPKL_id0000000014191666176
3FO4JXNTQPKL_id0000000014231666688
3FO4JXNTQPKL_id0000000014271666176
3FO4JXNTQPKL_id0000000014311666688
3FO4JXNTQPKL_id0000000014351666176
3FO4JXNTQPKL_id0000000014391666688
3FO4JXNTQPKL_id0000000014431666176
3FO4JXNTQPKL_id0000000014471666688
3FO4JXNTQPKL_id0000000014511666176
3FO4JXNTQPKL_id0000000014551666688
3FO4JXNTQPKL_id0000000014591666176
3FO4JXNTQPKL_id0000000014631666688
3FO4JXNTQPKL_id0000000014671666176
3FO4JXNTQPKL_id0000000014711666688
3FO4JXNTQPKL_id0000000014751666176
3FO4JXNTQPKL_id0000000014791666688
3FO4JXNTQPKL_id00000

Done (t=0.06s)
creating index...
index created!
3FO4JXN0A5L3_id0000000000031666668
3FO4JXN0A5L3_id0000000000071666664
3FO4JXN0A5L3_id0000000000111666664
3FO4JXN0A5L3_id0000000000151666672
3FO4JXN0A5L3_id0000000000191666672
3FO4JXN0A5L3_id0000000000231666672
3FO4JXN0A5L3_id0000000000271666656
3FO4JXN0A5L3_id0000000000311666656
3FO4JXN0A5L3_id0000000000351666656
3FO4JXN0A5L3_id0000000000391666656
3FO4JXN0A5L3_id0000000000431666656
3FO4JXN0A5L3_id0000000000471666656
3FO4JXN0A5L3_id0000000000511666656
3FO4JXN0A5L3_id0000000000551666688
3FO4JXN0A5L3_id0000000000591666688
3FO4JXN0A5L3_id0000000000631666688
3FO4JXN0A5L3_id0000000000671666688
3FO4JXN0A5L3_id0000000000711666688
3FO4JXN0A5L3_id0000000000751666688
3FO4JXN0A5L3_id0000000000791666688
3FO4JXN0A5L3_id0000000000831666688
3FO4JXN0A5L3_id0000000000871666688
3FO4JXN0A5L3_id0000000000911666688
3FO4JXN0A5L3_id0000000000951666688
3FO4JXN0A5L3_id0000000000991666688
3FO4JXN0A5L3_id0000000001031666688
3FO4JXN0A5L3_id0000000001071666688
3FO4JXN

Done (t=0.26s)
creating index...
index created!
3FO4JXKO1V6Q_id0000000000031666668
3FO4JXKO1V6Q_id0000000000071666664
3FO4JXKO1V6Q_id0000000000111666664
3FO4JXKO1V6Q_id0000000000151666672
3FO4JXKO1V6Q_id0000000000191666672
3FO4JXKO1V6Q_id0000000000231666672
3FO4JXKO1V6Q_id0000000000271666656
3FO4JXKO1V6Q_id0000000000311666656
3FO4JXKO1V6Q_id0000000000351666656
3FO4JXKO1V6Q_id0000000000391666656
3FO4JXKO1V6Q_id0000000000431666656
3FO4JXKO1V6Q_id0000000000471666656
3FO4JXKO1V6Q_id0000000000511666656
3FO4JXKO1V6Q_id0000000000551666688
3FO4JXKO1V6Q_id0000000000591666688
3FO4JXKO1V6Q_id0000000000631666688
3FO4JXKO1V6Q_id0000000000671666688
3FO4JXKO1V6Q_id0000000000711666688
3FO4JXKO1V6Q_id0000000000751666688
3FO4JXKO1V6Q_id0000000000791666688
3FO4JXKO1V6Q_id0000000000831666688
3FO4JXKO1V6Q_id0000000000871666688
3FO4JXKO1V6Q_id0000000000911666688
3FO4JXKO1V6Q_id0000000000951666688
3FO4JXKO1V6Q_id0000000000991666688
3FO4JXKO1V6Q_id0000000001031666688
3FO4JXKO1V6Q_id0000000001071666688
3FO4JXK

In [7]:
# def masks2instance_im(masks):
#     instance_im = np.zeros(shape=[masks.shape[0], masks.shape[1]])
#     for i in range(masks.shape[-1]):
#         instance_im[masks[...,i]] = (i + 1)
#     return instance_im

# image_ids = dataset_train.load_view(4)
# images = []
# masks = []
# for image_id in image_ids:
#     print(dataset_train.image_info[image_id]['path'])
#     image = dataset_train.load_image(image_id)
#     print(image_id)
#     mask, class_ids = dataset_train.load_mask(image_id)
# #     plt.imsave(os.path.join(ROOT_DIR, 'data/Results/{}_mask.png'.format(image_id)), masks2instance_im(mask))
# #     plt.imsave(os.path.join(ROOT_DIR, 'data/Results/{}.png'.format(image_id)), image)

#     im = visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
    

In [8]:
class TrainConfig(interior_transformer.InteriorNetConfig):
    TOP_DOWN_PYRAMID_SIZE = 72
    POST_NMS_ROIS_TRAINING = 500
    PRE_NMS_LIMIT = 1500
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    VALIDATION_STEPS = 20
    NUM_CLASSES = len(selected_classes)  # background + num classes
    samples = 1
    NUM_VIEWS = 2
    USE_RPN_ROIS = True
    LEARNING_RATE = 0.001
    BACKBONE = 'resnet50'
    VANILLA = False
    WEIGHT_DECAY = 0.0001
    TRANSFORMER = True

config_stage1 = TrainConfig()
config_stage1.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  640
IMAGE_META_SIZE                35
IMAGE_MIN_DIM                  640
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [640 640   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE              

In [9]:

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess= tf.Session(config=config)
keras.backend.tensorflow_backend.set_session(sess)

# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config_stage1,
                          model_dir=MODEL_DIR)
#model.load_weights(model_path, by_name=True)
print(model.keras_model.summary())
# Load pretrained weights
# model.load_weights(COCO_MODEL_PATH, by_name=True,
#                        exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
#                                 "mrcnn_bbox", "mrcnn_mask"])
# model.keras_model.save_weights(os.path.join(MODEL_DIR, 'interionet.h5'))

shape_norm_boxes_graph: Tensor("lambda_1/strided_slice:0", shape=(2,), dtype=int32)
shape_norm_boxes_graph: Tensor("lambda_1/strided_slice_1:0", shape=(2,), dtype=int32)
input_image_shape: [None, None, 640, 640, 3]
fn image_shape: [None, None, 640, 640, 3]
shape_out: [None, None, 160, 160, 72]
shape_out: [None, None, 80, 80, 72]
shape_out: [None, None, 40, 40, 72]
shape_out: [None, None, 20, 20, 72]
shape_out: [None, None, 10, 10, 72]
training transformer
feats shape: [None, None, 20, 20, 72]
Kmat shape: [None, 3, 3]
Kmat shape: [None, 2, 3, 3]
Rcam shape: [None, None, 3, 4]
image_indices shape: [1, 2, 3, 400]
Kmat shape: [None, 2, 3, 3]
camera_coordinates shape: [None, 2, None, None, None]
camera_coordinates shape: [1, 2, 1, 3, 400]
camera_coordinates shape: [1, 2, 1, 4, 400]
world_coordinates shape: [1, 2, 1, 3, 400]
feats shape: [None, None, 1, 20, 20, 72]
feats shape: [None, None, 20, 20, 72]
Kmat shape: [None, 3, 3]
Kmat shape: [None, 2, 3, 3]
Rcam shape: [None, None, 3, 4]
image_

In [10]:
layer = model.keras_model.get_layer('encoder_1')
print(layer.weights)

[<tf.Variable 'encoder_1/kernel:0' shape=(72, 72) dtype=float32_ref>, <tf.Variable 'encoder_1/bias:0' shape=(72,) dtype=float32_ref>, <tf.Variable 'encoder_1/kernel_1:0' shape=(72, 72) dtype=float32_ref>, <tf.Variable 'encoder_1/bias_1:0' shape=(72,) dtype=float32_ref>, <tf.Variable 'encoder_1/kernel_2:0' shape=(72, 72) dtype=float32_ref>, <tf.Variable 'encoder_1/bias_2:0' shape=(72,) dtype=float32_ref>, <tf.Variable 'encoder_1/kernel_3:0' shape=(72, 72) dtype=float32_ref>, <tf.Variable 'encoder_1/bias_3:0' shape=(72,) dtype=float32_ref>, <tf.Variable 'encoder_1/dense_6/kernel:0' shape=(72, 256) dtype=float32_ref>, <tf.Variable 'encoder_1/dense_6/bias:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'encoder_1/dense_7/kernel:0' shape=(256, 72) dtype=float32_ref>, <tf.Variable 'encoder_1/dense_7/bias:0' shape=(72,) dtype=float32_ref>, <tf.Variable 'encoder_1/kernel_4:0' shape=(72, 72) dtype=float32_ref>, <tf.Variable 'encoder_1/bias_4:0' shape=(72,) dtype=float32_ref>, <tf.Variable 'enc

In [11]:
# This schedule overwrites the learning rate in config
def step_decay(epoch):
        initial_lrate = 0.001
        drop = 0.5
        epochs_drop = 6
        lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
        if lrate < 0.001:
            return 0.001
        return lrate
lrate = LearningRateScheduler(step_decay)

callbacks_list = [lrate]

In [12]:
#log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=MODEL_DIR, histogram_freq=1)


In [13]:
# Add all callbacks to list
# from keras.backend.tensorflow_backend import set_session
# config_tf = tf.ConfigProto()
# config_tf.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
# config_tf.log_device_placement = True  # to log device placement (on which device the operation ran)
#                                     # (nothing gets printed in Jupyter, only if you run it standalone)
# sess = tf.Session(config=config_tf)
# set_session(sess)  # set this TensorFlow session as the default session for Keras

In [14]:
# from keras.engine import saving
# import h5py
# f = h5py.File(COCO_MODEL_PATH, mode='r')
# f = h5py.File(os.path.join(MODEL_DIR, 'interiornet20200330T1532', 'mask_rcnn_interiornet_0298.h5'), mode='r')
# for layer in model.keras_model.layers:
#     if layer.name == 'backbone':
#         layers = layer.layers
#         print(layer.__class__.__name__)
#         saving.load_weights_from_hdf5_group_by_name(f, layer.layers)
#         break

In [15]:
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
# model_path = os.path.join(MODEL_DIR, 'stage1', 'interior_sep.h5')
model_path = model.find_last()
#model_path =  COCO_MODEL_PATH
print(model_path)
# model.load_weights(model_path, by_name=True, exclude = ["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask", "grid_reas_P2/3D_conv", "grid_reas_P3/3D_conv", "grid_reas_P4/3D_conv", "grid_reas_P5/3D_conv", "grid_reas_P6/3D_conv"])
model.load_weights(model_path, by_name=True)
# model.load_weights(model_path, by_name=True, exclude = ["grid_reas_P2/3D_conv", "grid_reas_P3/3D_conv", "grid_reas_P4/3D_conv", "grid_reas_P5/3D_conv", "grid_reas_P6/3D_conv"])
# model.load_weights(model_path, by_name=True, exclude = ["backbone"])

FileNotFoundError: [Errno 2] Could not find weight files in /home/julius/Projects/MVOD/mulit_view_object_detection/logs/interiornet20200714T1532

In [ ]:
print("Train heads and grid reasoning layers")
# model.train(dataset_train, dataset_train,
#            learning_rate=config_stage1.LEARNING_RATE, custom_callbacks=callbacks_list,
#            epochs=20,
#            layers='all')
# model.train(dataset_train, dataset_val,
#             learning_rate=config_stage1.LEARNING_RATE,
#             epochs=20,
#             layers='all')

In [ ]:
# print("Train res5+")
# model.train(dataset_train, dataset_val,
#             learning_rate=config_stage1.LEARNING_RATE,
#             epochs=20,
#             layers='5+')

In [ ]:
# from keras.engine import saving
# import h5py
# #f = h5py.File(os.path.join(MODEL_DIR, '', 'backbone_sep.h5'), mode='r')
# for layer in model.keras_model.layers:
#     if layer.name == 'backbone':
#         layer.save_weights(os.path.join(MODEL_DIR, '', 'backbone_sep.h5'))
#         break

In [ ]:
image_id = 609
image_t, image_meta, gt_class_ids, gt_boxes, gt_masks = \
                    modellib.load_image_gt(dataset_train, config_stage1, image_id, augment=False,
                                  augmentation=None,
                                  use_mini_mask=config_stage1.USE_MINI_MASK)

In [ ]:
plt.imshow(image_t)
#plt.imshow(gt_masks[:,:,0])
print(gt_boxes)
print(image_meta)
print(gt_class_ids)

In [ ]:
a = modellib.data_generator(dataset_train, config_stage1, shuffle=True, augment=False, augmentation=None,
                   random_rois=0, batch_size=1, detection_targets=False,
                   no_augmentation_sources=None)
print(a)

In [ ]:
# inputs = [batch_images, batch_image_meta, batch_rpn_match, batch_rpn_bbox,
#                           batch_gt_class_ids, batch_gt_boxes, batch_gt_masks, batch_gt_R, batch_gt_Kmat]

inputs, output = next(a)
for i, input_i in enumerate(inputs):
    #print("{}'th input is: {}".format(i, input_i))
    print("shape of {}'ths input is: {}".format(i, input_i.shape))


In [ ]:
print(np.where(inputs[2]>0))